In [1]:
# dependencies BeautifulSoup, Pandas, and Requests/Splinter
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pymongo
import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\jjel0\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


### NASA Mars News

In [3]:
# Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text.
    # Assign the text to variables that you can reference later.
url_mars_news = 'https://redplanetscience.com/'
browser.visit(url_mars_news)
html_mars_news = browser.html

In [4]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_news = bs(html_mars_news, 'html.parser')

In [5]:
# Create a dictionary
data = {}

# Collect the latest News Title and Paragraph Text.
list_text_list = soup_mars_news.find('div', class_='list_text')

# Retrieve the title and article body.
content_title = list_text_list.find('div', class_='content_title').get_text()  
article_teaser_body = list_text_list.find('div', class_='article_teaser_body').get_text()

data["news_title"] = content_title
data["news_paragraph"] = article_teaser_body

data

{'news_title': "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes",
 'news_paragraph': "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. "}

### JPL Mars Space Images - Featured Image

In [6]:
# Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).
url_mars_imag = 'https://spaceimages-mars.com'
browser.visit(url_mars_imag)
html_mars_imag = browser.html

In [7]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_imag = bs(html_mars_imag, 'html.parser')

In [8]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url 
    # string to a variable called `featured_image_url`.  Make sure to find the image url to the full size `.jpg` image.
    # Example:
        # featured_image_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'
featured_image = soup_mars_imag.find('a', class_='showimg fancybox-thumbs')

# Create the image url
href = featured_image['href']
featured_image_url = 'https://spaceimages-mars.com/' + href
data

{'news_title': "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes",
 'news_paragraph': "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. "}

### Mars Facts

In [9]:
# Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing
    # facts about the planet including Diameter, Mass, etc.
url_mars_facts = 'https://galaxyfacts-mars.com'
browser.visit(url_mars_facts)
html_mars_facts = browser.html

In [10]:
# Create a Beautiful Soup object.
soup_mars_facts = bs(html_mars_facts, 'lxml')

In [11]:
# Use Pandas to convert the data to a HTML table string.
html_mars_facts_html = pd.read_html(url_mars_facts)[1]

html_mars_facts_html.columns = ["description", "value"]
data["facts"] = html_mars_facts_html.to_html(index=False)
data

{'news_title': "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes",
 'news_paragraph': "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. ",
 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>description</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperat

### Mars Hemispheres

In [12]:
# Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of 
    # Mars hemispheres.
url_mars_hemis = 'https://marshemispheres.com/'
browser.visit(url_mars_hemis)
html_mars_hemis = browser.html

In [13]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_hemis = bs(html_mars_hemis, 'html.parser')

# Create the hemis url list.
hemis_url_list = soup_mars_hemis.find_all('a', class_='itemLink product-item')
hemis_url_list

hemisphere_image_urls = []

# Loop through returned results to create the hemisphere images.
for i in range(4):
    browser.find_by_css('a.product-item img')[i].click()
    hemisphere = {}

    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css('h2.title').text
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    # Finally, we navigate backwards
    browser.back()

data["hemisphere_image_urls"] =  hemisphere_image_urls
data

{'news_title': "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes",
 'news_paragraph': "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. ",
 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>description</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperat

In [14]:
browser.quit()